In [1]:
import os
import sys
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")

import pandas as pd
import numpy as np
import datetime
import torch
import math

from Clust.clust.ML.regression_JS.ada_rnn import ada_rnn
from Clust.clust.preprocessing.imputation.basicMethod import BasicImputation
from Clust.clust.transformation.purpose.machineLearning import LSTMData

# Set Param

In [2]:
param = {
    "x_features" : 6, 
    "class_num" : 1, 
    "dropout" : 0.0, 
    "len_seq" : 24, 
    "loss_type" : "adv",
    "hidden_size" : 64,
    "num_layers" : 2
}

args = {
    "output_folder_name" : "./ada_rnn_model",
    "save_model_name" : "test_adarnn_model_by_origin_data",
    "lr" : 5e-4,
    "n_epochs" : 200,
    "num_layers" : 2, 
    "len_seq" : 24,
    "pre_epoch" : 40,
    "len_win" : 0,
    "dw" : 0.5,
    "early_stop" :40
}
k = 2

# get Data

In [3]:
# read original data
data_pickle = pd.read_pickle("original_data.pkl")
original_data = data_pickle["Dongsi"]

In [4]:
len(original_data), original_data[0].shape, original_data[2].shape

(3, (1460, 24, 6), (1460,))

In [5]:
# original data 에서 data_x, data_y 나누기
data_x = original_data[0]
data_y = original_data[2]

In [6]:
# set time
original_data_start_time = datetime.datetime(2013, 3, 1, 0, 0)

## train data time
train_start_time = '2013-03-01 00:00:00'
train_end_time = '2016-06-30 23:00:00'

## valid data time
valid_start_time='2016-7-2 00:00:00'
valid_end_time='2016-10-30 23:00:00'

In [7]:
# split train & validation data -> get train_x, train_y, valid_x, valid_y
index_start=(pd.to_datetime(train_start_time) - original_data_start_time).days
index_end=(pd.to_datetime(train_end_time) - original_data_start_time).days

train_x=data_x[index_start: index_end + 1]
train_y=data_y[index_start: index_end + 1]

index_start=(pd.to_datetime(valid_start_time) - original_data_start_time).days
index_end=(pd.to_datetime(valid_end_time) - original_data_start_time).days

valid_x=data_x[index_start: index_end + 1]
valid_y=data_y[index_start: index_end + 1]

In [8]:
train_x.shape, train_y.shape, valid_x.shape, valid_y.shape

((1218, 24, 6), (1218,), (121, 24, 6), (121,))

In [9]:
train_y

array([0.0013624 , 0.15258856, 0.08855586, ..., 0.14986376, 0.14032698,
       0.        ])

# Data Loader

In [10]:
adarnn = ada_rnn.ClustAdaRnn(param)

{'x_features': 6, 'class_num': 1, 'dropout': 0.0, 'len_seq': 24, 'loss_type': 'adv', 'hidden_size': 64, 'num_layers': 2}


In [11]:
train_loader_list, valid_loader = adarnn.create_trainloader(train_x, train_y, valid_x, valid_y, k, train_start_time, train_end_time)

# Training

In [12]:
## test loader 제거 작업 시작
adarnn.train(train_loader_list, valid_loader, args)

[2023-03-21 15:24:27] - create model...
[2023-03-21 15:24:27] - Epoch: 0
[2023-03-21 15:24:27] - training...


100%|██████████████████████████████████████████| 6/6 [00:00<00:00,  6.98it/s]

[2023-03-21 15:24:27] - evaluating...



Valid: 100%|██████████████████████████████████| 2/2 [00:00<00:00, 343.63it/s]

[2023-03-21 15:24:27] - train 0.132158, valid MSE Loss 0.032972, valid L1 Loss 0.161920
[2023-03-21 15:24:27] - Epoch: 1
[2023-03-21 15:24:27] - training...


...

[2023-03-21 15:25:07] - train 0.028142, valid MSE Loss 1.904612, valid L1 Loss 1.379972
[2023-03-21 15:25:07] - Epoch: 81
[2023-03-21 15:25:07] - training...



100%|██████████████████████████████████████████| 6/6 [00:00<00:00, 13.48it/s]

[2023-03-21 15:25:07] - evaluating...



Valid: 100%|██████████████████████████████████| 2/2 [00:00<00:00, 482.46it/s]

[2023-03-21 15:25:07] - train 0.025615, valid MSE Loss 1.579318, valid L1 Loss 1.256496
[2023-03-21 15:25:07] - early stop
[2023-03-21 15:25:07] - best val mse loss score: 0.0006846452306490391 @ 41
[2023-03-21 15:25:07] - Finished.
